<a href="https://colab.research.google.com/github/fbeilstein/machine_learning/blob/master/seminar_5_5_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
print("Hello")

Hello


Explore gradient descent with the following code.
You should understand how it works and its weaknesses.
If you run out of ideas, use the following parameters

function_name | x_ini | y_ini | max_iter | theta | comment
---|---|---|---|---|---
```x**2 + y**2``` | -5.0 | -4.0 | 20 | 0.1 | good convex function
```sin(x) + sin(y)``` | 1.0 | 1.0 | 10 | 0.01 | many global minima
```(1-x**2)+100*(y-x**2)**2``` | -1.0 | -1.0 | 10 | 0.01 | famous Rosenbrock function

In [58]:
#@title # Exploring gradient+hessian descent

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

#@markdown ---
#@markdown ##Gradient descent options
#@markdown These variables correspond to the parameters of the minimizator
x_ini = 1.9 #@param {type:"slider", min:-10.0, max:10.0, step:0.1}

y_ini = 2.4 #@param {type:"slider", min:-10.0, max:10.0, step:0.1}

max_iter = 50 #@param {type:"slider", min:2, max:50, step:1}

#@markdown ---
#@markdown ##Function to minimize
#@markdown Write function using sympy syntax.
#@markdown Use x and y as variables.
#@markdown You can use all elementary functions, (inverse)trigonometric, (inverse)hyperbolic functions, etc.
#@markdown for more details visit http://www.cfm.brown.edu/people/dobrush/am33/SymPy/function.html

function_name = 'x**2+(3*y)**2' #@param {type:"string"}

from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application, parse_expr
transformations = (standard_transformations + (implicit_multiplication_application,))
f = parse_expr(function_name, transformations=transformations)

from sympy import diff
Gx = diff(f, 'x')
Gy = diff(f, 'y')
g_xx = lambda x,y: diff(diff(f, 'x'), 'x').evalf(subs={'x':x, 'y':y})
g_xy = lambda x,y: diff(diff(f, 'x'), 'y').evalf(subs={'x':x, 'y':y})
g_yy = lambda x,y: diff(diff(f, 'y'), 'y').evalf(subs={'x':x, 'y':y})
H = lambda x,y: np.array([[g_xx(x, y), g_xy(x, y)], [g_xy(x, y), g_yy(x, y)]], dtype=float)

trace = [[x_ini, y_ini, f.evalf(subs={'x':x_ini, 'y':y_ini})]]
for i in range(max_iter):
  g_x = Gx.evalf(subs={'x':trace[-1][0], 'y':trace[-1][1]})
  g_y = Gy.evalf(subs={'x':trace[-1][0], 'y':trace[-1][1]})
  h = np.array([g_x, g_y])
  H_ = H(trace[-1][0], trace[-1][1])
  theta = np.dot(h, h) / np.dot(h, H_.dot(h))
  x_new = trace[-1][0] - theta * g_x
  y_new = trace[-1][1] - theta * g_y
  z_new = f.evalf(subs={'x':x_new, 'y':y_new})
  trace.append([x_new, y_new, z_new])
trace = np.array(trace, dtype=float)

x_min = min(-10.0, np.min(trace[:,0]))
x_max = max(10.0, np.max(trace[:,0]))
y_min = min(-10.0, np.min(trace[:,1]))
y_max = max(10.0, np.max(trace[:,1]))

# function calculated for 3d plot
x_ = np.linspace(x_min, x_max, num=50)
y_ = np.linspace(y_min, y_max, num=50)
z_ = np.array([[f.evalf(subs={'x':x__, 'y':y__}) for x__ in x_] for y__ in y_], dtype=float)

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scene"}, {"type": "xy"}]])

fig.add_trace(go.Scatter3d(x=trace[:, 0], y=trace[:, 1], z=trace[:, 2], 
                           marker=dict(size=4, colorscale='Viridis'),
                           line=dict(color='red', width=2)),
              row=1, col=1)

fig.add_trace(go.Surface(x=x_, y=y_, z=z_, opacity=0.9, showscale=False),
              row=1, col=1)

fig.add_trace(go.Contour(z=z_, x=x_, y=y_, contours=dict(showlabels=True)),
              row=1, col=2)

fig.add_trace(go.Scatter(x=trace[:, 0], y=trace[:, 1], line=dict(color='red', width=2)),
              row=1, col=2)

fig.update_layout(width=1200, height=600, autosize=False, 
                  title_text="Gradient descent demonstration",
                  #scene=dict(aspectratio = dict(x=1, y=1, z=1)),
                  showlegend=False)
fig.show()

In [0]:
#@title # Exploring Newton's method

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

#@markdown ---
#@markdown ##Gradient descent options
#@markdown These variables correspond to the parameters of the minimizator
x_ini = 1.9 #@param {type:"slider", min:-10.0, max:10.0, step:0.1}

y_ini = 2.4 #@param {type:"slider", min:-10.0, max:10.0, step:0.1}

max_iter = 50 #@param {type:"slider", min:2, max:50, step:1}

#@markdown ---
#@markdown ##Function to minimize
#@markdown Write function using sympy syntax.
#@markdown Use x and y as variables.
#@markdown You can use all elementary functions, (inverse)trigonometric, (inverse)hyperbolic functions, etc.
#@markdown for more details visit http://www.cfm.brown.edu/people/dobrush/am33/SymPy/function.html

function_name = 'x**2+(3*y)**2+sin(x)' #@param {type:"string"}

from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application, parse_expr
transformations = (standard_transformations + (implicit_multiplication_application,))
f = parse_expr(function_name, transformations=transformations)

from sympy import diff
Gx = diff(f, 'x')
Gy = diff(f, 'y')
g_xx = lambda x,y: diff(diff(f, 'x'), 'x').evalf(subs={'x':x, 'y':y})
g_xy = lambda x,y: diff(diff(f, 'x'), 'y').evalf(subs={'x':x, 'y':y})
g_yy = lambda x,y: diff(diff(f, 'y'), 'y').evalf(subs={'x':x, 'y':y})
H = lambda x,y: np.array([[g_xx(x, y), g_xy(x, y)], [g_xy(x, y), g_yy(x, y)]], dtype=float)

trace = [[x_ini, y_ini, f.evalf(subs={'x':x_ini, 'y':y_ini})]]
for i in range(max_iter):
  g_x = Gx.evalf(subs={'x':trace[-1][0], 'y':trace[-1][1]})
  g_y = Gy.evalf(subs={'x':trace[-1][0], 'y':trace[-1][1]})
  h = np.array([g_x, g_y])
  H_ = H(trace[-1][0], trace[-1][1])
  H_inv = np.linalg.inv(H_)
  theta = H_inv.dot(h)
  x_new = trace[-1][0] - theta[0]
  y_new = trace[-1][1] - theta[1]
  z_new = f.evalf(subs={'x':x_new, 'y':y_new})
  trace.append([x_new, y_new, z_new])
trace = np.array(trace, dtype=float)

x_min = min(-10.0, np.min(trace[:,0]))
x_max = max(10.0, np.max(trace[:,0]))
y_min = min(-10.0, np.min(trace[:,1]))
y_max = max(10.0, np.max(trace[:,1]))

# function calculated for 3d plot
x_ = np.linspace(x_min, x_max, num=50)
y_ = np.linspace(y_min, y_max, num=50)
z_ = np.array([[f.evalf(subs={'x':x__, 'y':y__}) for x__ in x_] for y__ in y_], dtype=float)

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scene"}, {"type": "xy"}]])

fig.add_trace(go.Scatter3d(x=trace[:, 0], y=trace[:, 1], z=trace[:, 2], 
                           marker=dict(size=4, colorscale='Viridis'),
                           line=dict(color='red', width=2)),
              row=1, col=1)

fig.add_trace(go.Surface(x=x_, y=y_, z=z_, opacity=0.9, showscale=False),
              row=1, col=1)

fig.add_trace(go.Contour(z=z_, x=x_, y=y_, contours=dict(showlabels=True)),
              row=1, col=2)

fig.add_trace(go.Scatter(x=trace[:, 0], y=trace[:, 1], line=dict(color='red', width=2)),
              row=1, col=2)

fig.update_layout(width=1200, height=600, autosize=False, 
                  title_text="Gradient descent demonstration",
                  #scene=dict(aspectratio = dict(x=1, y=1, z=1)),
                  showlegend=False)
fig.show()

In [37]:
#@title # Exploring simple gradient descent

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

#@markdown ---
#@markdown ##Gradient descent options
#@markdown These variables correspond to the parameters of the minimizator
x_ini = 7.6 #@param {type:"slider", min:-10.0, max:10.0, step:0.1}

y_ini = 1.7 #@param {type:"slider", min:-10.0, max:10.0, step:0.1}

max_iter = 50 #@param {type:"slider", min:2, max:50, step:1}

theta = 0.477 #@param {type:"slider", min:0, max:1, step:0.001}
#@markdown ---
#@markdown ##Function to minimize
#@markdown Write function using sympy syntax.
#@markdown Use x and y as variables.
#@markdown You can use all elementary functions, (inverse)trigonometric, (inverse)hyperbolic functions, etc.
#@markdown for more details visit http://www.cfm.brown.edu/people/dobrush/am33/SymPy/function.html

function_name = 'x**2+(3*y)**2' #@param {type:"string"}

from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application, parse_expr
transformations = (standard_transformations + (implicit_multiplication_application,))
f = parse_expr(function_name, transformations=transformations)

from sympy import diff
Gx = diff(f, 'x')
Gy = diff(f, 'y')

trace = [[x_ini, y_ini, f.evalf(subs={'x':x_ini, 'y':y_ini})]]
for i in range(max_iter):
  g_x = Gx.evalf(subs={'x':trace[-1][0], 'y':trace[-1][1]})
  g_y = Gy.evalf(subs={'x':trace[-1][0], 'y':trace[-1][1]})
  x_new = trace[-1][0] - theta * g_x
  y_new = trace[-1][1] - theta * g_y
  z_new = f.evalf(subs={'x':x_new, 'y':y_new})
  trace.append([x_new, y_new, z_new])
trace = np.array(trace, dtype=float)

x_min = min(-10.0, np.min(trace[:,0]))
x_max = max(10.0, np.max(trace[:,0]))
y_min = min(-10.0, np.min(trace[:,1]))
y_max = max(10.0, np.max(trace[:,1]))

# function calculated for 3d plot
x_ = np.linspace(x_min, x_max, num=50)
y_ = np.linspace(y_min, y_max, num=50)
z_ = np.array([[f.evalf(subs={'x':x__, 'y':y__}) for x__ in x_] for y__ in y_], dtype=float)

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scene"}, {"type": "xy"}]])

fig.add_trace(go.Scatter3d(x=trace[:, 0], y=trace[:, 1], z=trace[:, 2], 
                           marker=dict(size=4, colorscale='Viridis'),
                           line=dict(color='red', width=2)),
              row=1, col=1)

fig.add_trace(go.Surface(x=x_, y=y_, z=z_, opacity=0.9, showscale=False),
              row=1, col=1)

fig.add_trace(go.Contour(z=z_, x=x_, y=y_, contours=dict(showlabels=True)),
              row=1, col=2)

fig.add_trace(go.Scatter(x=trace[:, 0], y=trace[:, 1], line=dict(color='red', width=2)),
              row=1, col=2)

fig.update_layout(width=1200, height=600, autosize=False, 
                  title_text="Gradient descent demonstration",
                  #scene=dict(aspectratio = dict(x=1, y=1, z=1)),
                  showlegend=False)
fig.show()

In [42]:
#@title # Exploring minimization methods

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import minimize

#@markdown ---
#@markdown ##Minimization options
#@markdown These variables correspond to the parameters of the minimizator
x_ini = -7.1 #@param {type:"slider", min:-10.0, max:10.0, step:0.1}
y_ini = -3.6 #@param {type:"slider", min:-10.0, max:10.0, step:0.1}


method = 'Nelder-Mead' #@param ['Nelder-Mead','Powell','CG','BFGS','Newton-CG','L-BFGS-B','TNC','COBYLA','SLSQP','trust-constr','dogleg','trust-ncg','trust-exact','trust-krylov']

max_iter = 50 #@param {type:"slider", min:2, max:50, step:1}

jacobian = 'none' #@param ['none', '2-point', '3-point', 'cs', 'custom']

hessian = 'none' #@param ['none', '2-point', '3-point', 'cs', 'custom']

#@markdown ---
#@markdown ##Function to minimize
#@markdown Write function using sympy syntax.
#@markdown You can use all elementary functions, (inverse)trigonometric, (inverse)hyperbolic functions, etc.
#@markdown for more details visit http://www.cfm.brown.edu/people/dobrush/am33/SymPy/function.html


function_name = 'x**2+(3*y)**2' #@param {type:"string"}


from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application, parse_expr
transformations = (standard_transformations + (implicit_multiplication_application,))
f = parse_expr(function_name, transformations=transformations)

from sympy import diff
g_x = lambda x,y: diff(f, 'x').evalf(subs={'x':x, 'y':y})
g_y = lambda x,y: diff(f, 'y').evalf(subs={'x':x, 'y':y})

if jacobian == 'custom':
  jacobian = lambda x: np.array([g_x(x[0], x[1]), g_y(x[0], x[1])], dtype=float)
if jacobian == 'none':
  jacobian = None

g_xx = lambda x,y: diff(diff(f, 'x'), 'x').evalf(subs={'x':x, 'y':y})
g_xy = lambda x,y: diff(diff(f, 'x'), 'y').evalf(subs={'x':x, 'y':y})
g_yy = lambda x,y: diff(diff(f, 'y'), 'y').evalf(subs={'x':x, 'y':y})
H = lambda x,y: [[g_xx(x, y), g_xy(x, y)], [g_xy(x, y), g_yy(x, y)]]

if hessian == 'custom':
  hessian = lambda x: np.array(H(x[0], x[1]), dtype=float)
if hessian == 'none':
  hessian = None


func_to_minimize = lambda x: f.evalf(subs={'x':x[0], 'y':x[1]})
history = [np.array([x_ini, y_ini])]
store_data = lambda xk : history.append(xk)

minimize(func_to_minimize, [x_ini, y_ini], 
         method=method, jac=jacobian, hess=hessian,
         options={'maxiter':max_iter}, callback=store_data)

_h = np.array(history)
_x = _h.T[0]
_y = _h.T[1]
_z = np.array([func_to_minimize(x) for x in history], dtype=float)

x_min = min(-10.0, np.min(_x))
x_max = max(10.0, np.max(_x))
y_min = min(-10.0, np.min(_y))
y_max = max(10.0, np.max(_y))

x_ = np.linspace(x_min, x_max, num=50)
y_ = np.linspace(y_min, y_max, num=50)
z_ = np.array([[func_to_minimize([x,y]) for x in x_] for y in y_], dtype=float)

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scene"}, {"type": "xy"}]])

fig.add_trace(go.Scatter3d(x=_x, y=_y, z=_z, marker=dict(size=4, colorscale='Viridis'),
                                             line=dict(color='red', width=2)),
              row=1, col=1)

fig.add_trace(go.Surface(x=x_, y=y_, z=z_, opacity=0.9, showscale=False),
              row=1, col=1)

fig.add_trace(go.Contour(z=z_, x=x_, y=y_, contours=dict(showlabels=True)),
              row=1, col=2)

fig.add_trace(go.Scatter(x=_x, y=_y, line=dict(color='red', width=2)),
              row=1, col=2)

fig.update_layout(width=1200, height=600, autosize=False, 
                  title_text="Optimization demonstration",
                  scene=dict(aspectratio = dict(x=1, y=1, z=1)),
                  showlegend=False)
fig.show()